In [ ]:
%cd ..

In [ ]:
import numpy as np
import pandas as pd
import random
import os
#import keras_tuner as kt
from scipy import signal
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as layers
from tensorflow.keras.layers import GlobalMaxPooling2D, Activation, Dense, Conv1D, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalMaxPooling1D
from tensorflow.keras import optimizers

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras import regularizers



from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from utils.signal import DCFilter, normalize
from utils.visualize import showMe, showHistory, showAvg
from utils.augment import apply_augment


from utils.models.regression.conv2 import get_model

from utils.data import load_set, pre_process_set, format2train, shuffle
from utils.sampler_generator import SamplerGenerator

from utils.visualizer import Visualizer

%load_ext autoreload
%autoreload 2



from config.fist import settings

np.random.seed(42)
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [ ]:
end_label = True
train_generator = SamplerGenerator(settings, batch_size=settings['batch_size'], n_steps = 500, end_label=end_label)
val_generator = SamplerGenerator(settings, batch_size=settings['batch_size'], n_steps = 500, end_label=end_label, split = 'val')

In [ ]:
#from utils.models.regression.dense import get_model
from utils.models.regression.lstm import get_model

model = get_model(end_label)

In [ ]:

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=20,
    decay_rate=0.98
    )
    

loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=True,
    label_smoothing=0.005,
    axis=-1,
    #reduction="auto",
    name="binary_crossentropy",
)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)
#opt = keras.optimizers.Adam()


model.compile(optimizer=opt,
          loss=loss,
          metrics=['accuracy']
         )

In [ ]:
history = model.fit(train_generator,
                    validation_data=val_generator,
                    batch_size=settings['batch_size'],
                    epochs=100
                  )

showHistory(history)          